In [46]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score

df = pd.read_csv('df.csv')
df = df.drop(columns = ['NObeyesdad', 'BMI', 'Outliers', 'Weight'])

df.shape

(2107, 16)

In [47]:
df

Gender        Age    Height family_history_with_overweight FAVC  FCVC  \
0     Female  21.000000  1.620000                            yes   no   2.0   
1     Female  21.000000  1.520000                            yes   no   3.0   
2       Male  23.000000  1.800000                            yes   no   2.0   
3       Male  27.000000  1.800000                             no   no   3.0   
4       Male  22.000000  1.780000                             no   no   2.0   
...      ...        ...       ...                            ...  ...   ...   
2102  Female  20.976842  1.710730                            yes  yes   3.0   
2103  Female  21.982942  1.748584                            yes  yes   3.0   
2104  Female  22.524036  1.752206                            yes  yes   3.0   
2105  Female  24.361936  1.739450                            yes  yes   3.0   
2106  Female  23.664709  1.738836                            yes  yes   3.0   

      NCP       CAEC SMOKE      CH2O  SCC       FAF       TUE        CALC  \
0     3.0  Sometimes    no  2.000000   no  0.000000  1.000000          no   
1     3.0  Sometimes   yes  3.000000  yes  3.000000  0.000000   Sometimes   
2     3.0  Sometimes    no  2.000000   no  2.000000  1.000000  Frequently   
3     3.0  Sometimes    no  2.000000   no  2.000000  0.000000  Frequently   
4     1.0  Sometimes    no  2.000000   no  0.000000  0.000000   Sometimes   
...   ...        ...   ...       ...  ...       ...       ...         ...   
2102  3.0  Sometimes    no  1.728139   no  1.676269  0.906247   Sometimes   
2103  3.0  Sometimes    no  2.005130   no  1.341390  0.599270   Sometimes   
2104  3.0  Sometimes    no  2.054193   no  1.414209  0.646288   Sometimes   
2105  3.0  Sometimes    no  2.852339   no  1.139107  0.586035   Sometimes   
2106  3.0  Sometimes    no  2.863513   no  1.026452  0.714137   Sometimes   

                     MTRANS        Level  
0     Public_Transportation       Normal  
1     Public_Transportation       Normal  
2     Public_Transportation       Normal  
3                   Walking   Overweight  
4     Public_Transportation   Overweight  
...                     ...          ...  
2102  Public_Transportation  Obesity_III  
2103  Public_Transportation  Obesity_III  
2104  Public_Transportation  Obesity_III  
2105  Public_Transportation  Obesity_III  
2106  Public_Transportation  Obesity_III  

[2107 rows x 16 columns]

In [52]:

## XGBoost


X = df.drop('Level', axis = 1)
y = df['Level']
X = pd.get_dummies(X)
le = LabelEncoder()
le.fit(y)


le.classes_

y = le.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


params = {
            'objective':'multi:softprob',
            'max_depth': 5,
            'learning_rate': 0.1,
            'n_estimators':160
        }         
           

# instantiate the classifier 
xgb_clf = XGBClassifier(**params)


# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)
accuracy_score(y_pred, y_test)

xgb_kappa_score = cohen_kappa_score(y_test, y_pred)
print('Kappa Score: ',  xgb_kappa_score)


Kappa Score:  0.8265385116017785


In [49]:
le.classes_

array(['Normal', 'Obesity_I', 'Obesity_II', 'Obesity_III', 'Overweight',
       'Underweight'], dtype=object)

In [50]:
cm_xgb = confusion_matrix(y_test, y_pred)
cm_xgb

array([[46,  3,  0,  0, 12,  4],
       [ 6, 56,  3,  0,  6,  0],
       [ 2,  1, 69,  3,  1,  0],
       [ 0,  0,  1, 48,  0,  0],
       [10,  3,  1,  0, 94,  0],
       [ 2,  0,  0,  0,  2, 49]], dtype=int64)

In [51]:
import seaborn as sns
import matplotlib.pyplot as plt     
sns.set_theme(rc={'figure.figsize':(10.5,6)})
ax= plt.subplot()
sns.heatmap(cm_xgb, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation





# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Normal', 'ObesityI', 'ObesityII', 'ObesityIII', 'OverWT',
       'UnderWT']); ax.yaxis.set_ticklabels(['Normal', 'ObesityI', 'ObesityII', 'ObesityIII', 'OverWT',
       'UnderWT']);

In [45]:
## RForests


X = df.drop('Level', axis = 1)
y = df['Level']
X = pd.get_dummies(X)
le = LabelEncoder()
le.fit(y)


le.classes_

y = le.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





params = {
           'n_estimators':20,
            'max_depth': 30,
            'max_features': 'sqrt',
            'n_estimators':100,
            'min_samples_leaf': 1, 
            'min_samples_split': 10

        }         
           

# instantiate the classifier 
xgb_clf = RandomForestClassifier(**params)


# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)
accuracy_score(y_pred, y_test)

cm_rf = confusion_matrix(y_test, y_pred)
cm_rf

rf_kappa_score = cohen_kappa_score(y_test, y_pred)
print('Kappa Score: ',  rf_kappa_score)

Kappa Score:  0.7916235177933381


In [39]:
import seaborn as sns
import matplotlib.pyplot as plt     
sns.set_theme(rc={'figure.figsize':(10.5,6)})
ax= plt.subplot()
sns.heatmap(cm_rf, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation





# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Normal', 'ObesityI', 'ObesityII', 'ObesityIII', 'OverWT',
       'UnderWT']); ax.yaxis.set_ticklabels(['Normal', 'ObesityI', 'ObesityII', 'ObesityIII', 'OverWT',
       'UnderWT']);

In [40]:
y

array([0, 0, 0, ..., 3, 3, 3])

In [41]:

le.classes_

array(['Normal', 'Obesity_I', 'Obesity_II', 'Obesity_III', 'Overweight',
       'Underweight'], dtype=object)

In [42]:
df

Gender        Age    Height family_history_with_overweight FAVC  FCVC  \
0     Female  21.000000  1.620000                            yes   no   2.0   
1     Female  21.000000  1.520000                            yes   no   3.0   
2       Male  23.000000  1.800000                            yes   no   2.0   
3       Male  27.000000  1.800000                             no   no   3.0   
4       Male  22.000000  1.780000                             no   no   2.0   
...      ...        ...       ...                            ...  ...   ...   
2102  Female  20.976842  1.710730                            yes  yes   3.0   
2103  Female  21.982942  1.748584                            yes  yes   3.0   
2104  Female  22.524036  1.752206                            yes  yes   3.0   
2105  Female  24.361936  1.739450                            yes  yes   3.0   
2106  Female  23.664709  1.738836                            yes  yes   3.0   

      NCP       CAEC SMOKE      CH2O  SCC       FAF       TUE        CALC  \
0     3.0  Sometimes    no  2.000000   no  0.000000  1.000000          no   
1     3.0  Sometimes   yes  3.000000  yes  3.000000  0.000000   Sometimes   
2     3.0  Sometimes    no  2.000000   no  2.000000  1.000000  Frequently   
3     3.0  Sometimes    no  2.000000   no  2.000000  0.000000  Frequently   
4     1.0  Sometimes    no  2.000000   no  0.000000  0.000000   Sometimes   
...   ...        ...   ...       ...  ...       ...       ...         ...   
2102  3.0  Sometimes    no  1.728139   no  1.676269  0.906247   Sometimes   
2103  3.0  Sometimes    no  2.005130   no  1.341390  0.599270   Sometimes   
2104  3.0  Sometimes    no  2.054193   no  1.414209  0.646288   Sometimes   
2105  3.0  Sometimes    no  2.852339   no  1.139107  0.586035   Sometimes   
2106  3.0  Sometimes    no  2.863513   no  1.026452  0.714137   Sometimes   

                     MTRANS        Level  
0     Public_Transportation       Normal  
1     Public_Transportation       Normal  
2     Public_Transportation       Normal  
3                   Walking   Overweight  
4     Public_Transportation   Overweight  
...                     ...          ...  
2102  Public_Transportation  Obesity_III  
2103  Public_Transportation  Obesity_III  
2104  Public_Transportation  Obesity_III  
2105  Public_Transportation  Obesity_III  
2106  Public_Transportation  Obesity_III  

[2107 rows x 16 columns]

0.7827579106321642